# 🌐 PYUSD Advanced Monitoring
Includes:
- Ethereum (real-time + execution trace via `debug_traceTransaction`, `trace_block`)
- Solana support (via public RPC)
- Google Sheets integration


In [ ]:
# 📦 Install required packages
!pip install --quiet web3 solana gspread gspread_dataframe google-auth pandas requests

In [ ]:
# 📚 Imports and Auth
from web3 import Web3
import requests
from solana.rpc.api import Client as SolanaClient
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth

# Authenticate
auth.authenticate_user()
import google.auth
creds, _ = google.auth.default()
gc = gspread.authorize(creds)
print("✅ Authenticated")

In [ ]:
# 🔌 Ethereum GCP Node Setup
GCP_RPC_URL = 'https://<your-node>.blockchainnodeengine.googleapis.com'  # Replace with your actual node
web3 = Web3(Web3.HTTPProvider(GCP_RPC_URL))
PYUSD_CONTRACT = '0x6c3ea9036406852006290770bedfcaba0e23a0e8'
TRANSFER_TOPIC = web3.keccak(text="Transfer(address,address,uint256)").hex()
print("🔗 Connected to Ethereum via GCP")

In [ ]:
# 🔎 Ethereum Debug Trace (example: latest block)
def trace_block(block_number):
    payload = {
        "jsonrpc": "2.0",
        "method": "trace_block",
        "params": [hex(block_number)],
        "id": 1
    }
    headers = {'Content-Type': 'application/json'}
    response = requests.post(GCP_RPC_URL, headers=headers, json=payload)
    return response.json()

block_number = web3.eth.block_number - 1
trace_data = trace_block(block_number)
trace_data

In [ ]:
# 🔍 Ethereum TraceTransaction (example tx hash required)
def trace_transaction(tx_hash):
    payload = {
        "jsonrpc": "2.0",
        "method": "debug_traceTransaction",
        "params": [tx_hash],
        "id": 1
    }
    headers = {'Content-Type': 'application/json'}
    response = requests.post(GCP_RPC_URL, headers=headers, json=payload)
    return response.json()

# Example usage:
# trace_transaction('0x123...')

In [ ]:
# 🌞 Solana RPC Setup
solana = SolanaClient("https://api.mainnet-beta.solana.com")
pyusd_solana_mint = "Es9vMFrzaCERmFdGJ7aPKfEWZTz2xgZ6EoH6wUK1Vq5X"  # Replace with actual PYUSD if different

def get_recent_pyusd_transfers():
    result = solana.get_signatures_for_address(pyusd_solana_mint, limit=10)
    if result['result']:
        return pd.DataFrame(result['result'])
    return pd.DataFrame()

sol_df = get_recent_pyusd_transfers()
sol_df.head()

In [ ]:
# 📤 Export both Ethereum and Solana data to Google Sheets
sheet = gc.create("PYUSD_Multichain_Analytics")
eth_ws = sheet.get_worksheet(0)
set_with_dataframe(eth_ws, pd.DataFrame(trace_data.get('result', [])))

# Create new worksheet for Solana
sol_ws = sheet.add_worksheet(title="Solana_PYUSD", rows="100", cols="20")
set_with_dataframe(sol_ws, sol_df)

print("✅ Exported to Google Sheets:", sheet.url)